In [199]:
import numpy as np
import pandas as pd
import rdflib
import json
import xml
import os

import xml.etree.ElementTree as Tree

# Load data

In [18]:
!ls ~/data/waste-disposal-schleswig-holstein/

Q_II_10_j15_SH_korr.xlsx
Q_II_10_j16_SH_korr.xlsx
Q_II_10_j17_SH.xlsx
abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2016-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2017-anlagen-der-abfallentsorgung-korrektur.rdf


In [82]:
!head -n10 ~/data/waste-disposal-schleswig-holstein/abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:locn="http://www.w3.org/ns/locn#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:dcat="http://www.w3.org/ns/dcat#"
  xmlns:dct="http://purl.org/dc/terms/"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:vcard="http://www.w3.org/2006/vcard/ns#"
  xmlns:dcatde="http://dcat-ap.de/def/dcatde/"


In [160]:
DATA_PATH = os.path.join(
    os. environ['HOME'],
    'data/waste-disposal-schleswig-holstein/')
YEARS = [2015,2016,2017]
FILENAME = 'abfallentsorgung-in-schleswig-holstein-$-anlagen-der-abfallentsorgung-korrektur.rdf'

In [205]:
def load_rdf(year):
    XPATH_DATASET = '{http://www.w3.org/ns/dcat#}Dataset[1]' 
    XPATH_SPATIAL = '{http://purl.org/dc/terms/}spatial[1]'
    XPATH_LOCATION = '{http://purl.org/dc/terms/}Location[1]'
    XPATH_GEOMETRY = '{http://www.w3.org/ns/locn#}geometry'
    XPATH_ATTR_DATATYPE = '{http://www.w3.org/1999/02/22-rdf-syntax-ns#}datatype'
    OPENGIS = 'https://www.iana.org/assignments/media-types/application/vnd.geo+json'
    
    filename = FILENAME.replace('$', str(year))
    tree = Tree.parse(os.path.join(DATA_PATH, filename))
    node = tree.getroot().findall('{}/{}/{}/{}[@{}="{}"]'.format(
        XPATH_DATASET, 
        XPATH_SPATIAL,
        XPATH_LOCATION,
        XPATH_GEOMETRY,
        XPATH_ATTR_DATATYPE,
        OPENGIS))[0]
    jsondata = json.loads(node.text)
    coords = jsondata['coordinates'] # Type: OpenGIS - Multipolygon
    return (year, coords)

annual_coords = map(load_rdf, YEARS)

# Parse Coords into Pandas

In [163]:
cd = next(annual_coords)
v = rdf_to_pandas(cd)

KeyError: 'http'

In [145]:
v[0].attrib

{'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}datatype': 'https://www.iana.org/assignments/media-types/application/vnd.geo+json'}

In [48]:
dfcoord = pd.concat(map(rdf_to_pandas, annual_coords))